In [ ]:
# ## KAGGLE ONLY
from shutil import copyfile
copyfile(src="../input/inputs/generate_dataloaders.py", dst="../working/generate_dataloaders.py")
copyfile(src="../input/eval-script6/evaluation.py", dst="../working/evaluation.py")
copyfile(src="../input/inputs/train_dataloader.p", dst="../working/train_dataloader.p")
copyfile(src="../input/inputs/centroids_dataloader.p", dst="../working/ground_truth_dataloader.p")
copyfile(src="../input/dictionary/dictionary_2.p", dst="../working/dictionary.p")

copyfile(src="../input/bert-inputs/bert_train_unlabeled_dataloader.p", dst="../working/train_unlabeled_dataloader.p")
copyfile(src="../input/bert-inputs/bert_train_labeled_dataloader.p", dst="../working/train_labeled_dataloader.p")
copyfile(src="../input/bert-inputs/bert_val_dataloader.p", dst="../working/val_dataloader.p")

!pip install transformers

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *

from tqdm import tqdm_notebook as tqdm

import evaluation
import importlib
importlib.reload(evaluation)

In [ ]:
#Import bert stuff

from transformers import (
    BertModel,
    BertTokenizer
)

## Get Dataloaders

In [ ]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [ ]:
path = os.getcwd()
data_dir = path + '/'
# data_dir = path +'/data/' #Uncomment for local system

#### *Verify filenames are consistent*

In [ ]:
train_loader_labelled = pkl.load(open(data_dir + 'train_labeled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabeled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))

In [ ]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [ ]:
print(torch.__version__)

## Defining a Bert Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [ ]:
class BERTSequenceClassifier(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.bert = BertModel.from_pretrained('bert-base-cased', output_attentions=True)
        self.X = nn.Linear(bert.config.hidden_size, 10)
        self.W = nn.Linear(10, num_classes)
        self.num_classes = num_classes
        
    def forward(self, input_ids, attention_mask, token_type_ids):
        h, _, attn = self.bert(input_ids=input_ids, 
                               attention_mask=attention_mask, 
                               token_type_ids=token_type_ids)
        h_cls = h[:, 0]
        X_output = F.relu(self.X(h_cls))
        
        logits = self.W(X_output)
        
        return logits, attn

## Let's print what bert's architecture looks like

In [ ]:
bert = BertModel.from_pretrained('bert-base-cased', output_attentions=True)

In [ ]:
bert.parameters

## First performing fully supervised learning using the labelled set to train new vector representations

In [ ]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

model = BERTSequenceClassifier(num_classes = 2).to(current_device)

In [ ]:
criterion = nn.CrossEntropyLoss(reduction='sum',ignore_index=-1).to(current_device)
optimizer = optim.Adam([p for p in model.parameters() if p.requires_grad], lr=2e-05, eps=1e-08, amsgrad = True)

## Supervised model training

In [ ]:
def train_supervised_model(model, criterion, optimizer, train_loader_labelled, valid_loader, num_epochs=10, path_to_save=None, print_every = 1000):

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    for epoch in range(num_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        total_epoch_loss = 0
        
        for i,(input_ids_labelled, attention_mask_labelled, token_type_ids_labelled, labels) in tqdm(enumerate(train_loader_labelled)):
            
            input_ids_labelled = input_ids_labelled.to(current_device)
            attention_mask_labelled = attention_mask_labelled.to(current_device)
            token_type_ids_labelled = token_type_ids_labelled.to(current_device)
            labels = labels.to(current_device)

            # forward pass and compute loss
            logits, attn = model(input_ids_labelled, attention_mask_labelled, token_type_ids_labelled)
            
            loss = criterion(logits, labels)
        
            # run update step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += loss.data

            if i % print_every == 0:
                losses = loss/len(input_ids_labelled)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader_labelled.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(valid_loader):
            model.eval()
            
            input_ids = input_ids.to(current_device)
            attention_mask = attention_mask.to(current_device)
            token_type_ids = token_type_ids.to(current_device)
            labels = labels.to(current_device)
            
            # forward pass and compute loss
            logits,attn = model(input_ids, attention_mask, token_type_ids)
            
            loss = criterion(logits, labels)
            
            #Add loss to the validation loss
            total_validation_loss += loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"num_classes":model.num_classes}
            torch.save(model.state_dict(), path_to_save+'model_dict_labelled.pt')
            torch.save(train_losses, path_to_save+'train_losses_labelled')
            torch.save(val_losses, path_to_save+'val_losses_labelled')
            torch.save(opts, path_to_save+'opts_labelled')
        
    return model, train_losses, val_losses

In [ ]:
path = os.getcwd()
# model_folder = 'bert_model/'
# model_dir = path + '/models/' + model_folder
model_dir = path

In [ ]:
len(train_loader_labelled)

In [ ]:
model, train_losses, val_losses = train_supervised_model(model, criterion, optimizer, train_loader_labelled, val_loader, num_epochs=4, path_to_save=model_dir)


### Clustering Stuff

In [ ]:
class KMeansCriterion(nn.Module):
    
    def __init__(self, lmbda):
        super().__init__()
        self.lmbda = lmbda
    
    def forward(self, embeddings, centroids, labelled = False,  cluster_assignments = None):
        if labelled:
            num_reviews = len(cluster_assignments)
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances = distances[list(range(num_reviews)),cluster_assignments]
            loss = self.lmbda * cluster_distances.sum()
        else:
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances, cluster_assignments = distances.min(1)
            loss = self.lmbda * cluster_distances.sum()
        return loss, cluster_assignments

In [ ]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (input_ids, attention_mask, token_type_ids, labels) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(input_ids.to(current_device), attention_mask.to(current_device), token_type_ids.to(current_device))
    
        update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                        cluster_assignments.detach(), sentence_embed[0].to(current_device).detach())
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)

## Dataloader stuff

In [ ]:
def loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled):
    try:
        input_ids, attention_mask, token_type_ids, labels = next(train_loader_labelled_iter)
    except StopIteration:
        train_loader_labelled_iter = iter(train_loader_labelled)
        input_ids, attention_mask, token_type_ids, labels = next(train_loader_labelled_iter)

    return input_ids, attention_mask, token_type_ids, labels, train_loader_labelled_iter


def loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled):
    try:
        input_ids, attention_mask, token_type_ids, labels = next(train_loader_unlabelled_iter)
    except StopIteration:
        train_loader_unlabelled_iter = iter(train_loader_unlabelled)
        input_ids, attention_mask, token_type_ids, labels = next(train_loader_unlabelled_iter)

    return input_ids, attention_mask, token_type_ids, labels, train_loader_unlabelled_iter

### Training Function

In [ ]:
def train_clusters(model, centroids, criterion, optimizer, train_loader_labelled, train_loader_unlabelled, valid_loader, num_epochs=10, num_batches = 1000, path_to_save=None, print_every = 1000):

    train_loader_labelled_iter = iter(train_loader_labelled)
    train_loader_unlabelled_iter = iter(train_loader_unlabelled)

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    for epoch in range(num_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.eval() # we're only clustering, not training model
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0
        
        for i in tqdm(range(int(num_batches))):
            input_ids_labelled, attention_mask_labelled, token_type_ids_labelled, labels, train_loader_labelled_iter = loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled)
            input_ids_unlabelled, attention_mask_unlabelled, token_type_ids_unlabelled, _, train_loader_unlabelled_iter = loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled)

            input_ids_labelled = input_ids_labelled.to(current_device)
            attention_mask_labelled = attention_mask_labelled.to(current_device)
            token_type_ids_labelled = token_type_ids_labelled.to(current_device)
            labels = labels.to(current_device)
            
            input_ids_unlabelled = input_ids_unlabelled.to(current_device)
            attention_mask_unlabelled = attention_mask_unlabelled.to(current_device)
            token_type_ids_unlabelled = token_type_ids_unlabelled.to(current_device)

            # forward pass and compute loss
            sentence_embed_labelled,attn = model(input_ids_labelled, attention_mask_labelled, token_type_ids_labelled)
            sentence_embed_unlabelled,attn = model(input_ids_unlabelled, attention_mask_unlabelled, token_type_ids_unlabelled)
            
            cluster_loss_unlabelled, cluster_assignments_unlabelled = criterion(sentence_embed_unlabelled, centroids.detach())
            cluster_loss_labelled, cluster_assignments_labelled = criterion(sentence_embed_labelled, centroids.detach(), labelled = True, cluster_assignments = labels)
    
            total_batch_loss = cluster_loss_labelled.data + cluster_loss_unlabelled.data
            
#             #Add loss to the epoch loss
            total_epoch_loss += total_batch_loss.data

#             # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_labelled.detach(), sentence_embed_labelled.detach())
    
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_unlabelled.detach(), sentence_embed_unlabelled.detach())

            if i % print_every == 0:
                losses = total_batch_loss/(len(input_ids_labelled)+ len(input_ids_unlabelled))
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= (len(train_loader_labelled.dataset)+len(train_loader_unlabelled.dataset))
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (input_ids, attention_mask, token_type_ids, labels) in enumerate(valid_loader):
            model.eval()
            input_ids = input_ids.to(current_device)
            attention_mask = attention_mask.to(current_device)
            token_type_ids = token_type_ids.to(current_device)
            labels = labels.to(current_device)
            
            # forward pass and compute loss
            sentence_embed,attn = model(input_ids, attention_mask, token_type_ids)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"num_classes":model.num_classes}
            torch.save(model.state_dict(), path_to_save+'model_dict_unlabelled.pt')
            torch.save(centroids, path_to_save+'centroids_unlabelled')
            torch.save(train_losses, path_to_save+'train_losses_unlabelled')
            torch.save(val_losses, path_to_save+'val_losses_unlabelled')
            torch.save(opts, path_to_save+'opts_unlabelled')
            
        
    return model, centroids, train_losses, val_losses

In [ ]:
unsupervised_model = model
unsupervised_model.W = nn.Identity()

In [ ]:
# centroids = centroid_init(2, unsupervised_model.bert.config.hidden_size, train_loader_labelled, unsupervised_model, current_device)
centroids = centroid_init(2, 10, train_loader_labelled, unsupervised_model, current_device)
criterion = KMeansCriterion(1).to(current_device)
optimizer = optim.Adam([p for p in unsupervised_model.parameters() if p.requires_grad], lr=2e-05, eps=1e-08, amsgrad = True)

In [ ]:
centroids.shape

In [ ]:
path = os.getcwd()
# model_folder = 'bert_model/'
# model_dir = path + '/models/' + model_folder
model_dir = path

In [ ]:
num_batches = int(len(train_loader_unlabelled.dataset)/train_loader_unlabelled.batch_size)+1
num_batches

In [ ]:
unsupervised_model, bert_centroids, bert_train_losses, bert_val_losses = train_clusters(unsupervised_model, centroids, criterion, optimizer, train_loader_labelled,train_loader_unlabelled, val_loader, num_epochs=2, num_batches=num_batches, path_to_save=model_dir)


In [ ]:
torch.save(bert_centroids, model_dir+'centroids_unlabelled')

In [ ]:
# #Only needed for Kaggle

# from IPython.display import FileLink, FileLinks 
# FileLinks('.') #lists all downloadable files on server

# Evaluate Model

## Supervised Evaluation

In [ ]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

In [ ]:
## This cell will change for each model
model_folder = 'bert_model/'

criterion = nn.CrossEntropyLoss(reduction='sum')
criterion = criterion.to(current_device)

path = os.getcwd()
# model_dir = path + '/models/' + model_folder
model_dir = path

opts = torch.load(model_dir+'opts_labelled')
model = BERTSequenceClassifier(opts['num_classes']) #change here depending on model
model.load_state_dict(torch.load(model_dir+'model_dict_labelled.pt',map_location=lambda storage, loc: storage))
model = model.to(current_device)

In [ ]:
empty_centroids = torch.tensor([])

TP_cluster, FP_cluster=evaluation.bert(model, empty_centroids, val_loader, criterion, data_dir, current_device)

In [ ]:
TP_cluster[TP_cluster["original"] == 0]

## Unsupervised Evaluation

In [ ]:
## This cell will change for each model
model_folder = 'bert_model/'

criterion = KMeansCriterion(1)
criterion = criterion.to(current_device)

path = os.getcwd()
# model_dir = path + '/models/' + model_folder
model_dir = path

opts = torch.load(model_dir+'opts_labelled')
model = BERTSequenceClassifier(opts['num_classes']) #change here depending on model
model.load_state_dict(torch.load(model_dir+'model_dict_labelled.pt',map_location=lambda storage, loc: storage))
model = model.to(current_device)
model.W = nn.Identity()
centroids = torch.load(model_dir+'centroids_unlabelled',map_location=lambda storage, loc: storage)
centroids = centroids.to(current_device)


In [ ]:
TP_cluster, FP_cluster=evaluation.bert(model, centroids, val_loader, criterion, data_dir, current_device)

In [ ]:
TP_clusterer

In [ ]:
#FP_cluster

In [ ]:
#FP_cluster[FP_cluster.original == 1]